# ROS Navigation

## Chapter 6: Obstacle Avoidance

<table style="width:100%">
  <tr>
    <th>Estimated time of completion: 1'5 hours<br><br>What you will learn with this unit?
<ul>
  <li>What means Obstacle Avoidance in ROS Navigation?</li>
  <li>How does Obstacle Avoidance work?</li>
  <li>How do we perform Obstacle Avoidance in ROS?</li>
</ul></th>
    <th><img src="img/husky.png" width="212" height="236" /></th> 
  </tr>
</table>

And finally... here it is. The last Chapter of the Course! But I said last, not least! In fact, Obastacle Avoidance is one of the most important parts of ROS Navigation. What's the meaning of an autonomous Navigation, if your robot can't even avoid objects? None, right? 
<br>
Most of the environments you'll want your robot to Navigate will have obstacles. Tables, chairs, walls, even people walking in eventually. So you'll need to know how to avoid them.
<br><br>
And this is where this last Chapter comes to your rescue! But before starting with the core of the Chapter, let's have a look at how RViz can help us in this process, as we've done in all previous Chapters.

## Visualize Obstacle Avoidance in Rviz

As you've already seen in previous chapters, you can also launch RViz and add displays in order to watch the Obstacle Avoidance process of the robot. For this chapter, you'll basically need to use 3 elements of RViz.

### Visualize Local Costmap

<ol>
<li>Click the Add button under Displays and chose the Map element.</li> 
<li>Set the topic to /move_base/local_costmap/costmap</li>
</ol>

### Visualize Local Plan

<ol>
<li>Click the Add button under Displays and chose the Plan element.</li> 
<li>Set the topic to /move_base/DWAPlannerROS/local_plan</li>
</ol>

### 2D Elements

Explained in the previous Chapter

## Path Planning vs. Obstacle Avoidance

First of all, in order to understand what exactly is the Obstacle Avoidance and how it fits in the whole Navigation System, I think it's important that you understand the difference between Path Planning and Obstacle Avoidance. In order to accomplish this, let's do a very interesting exercise that I think will help a lot to this purpose.

<p style="background:#EE9023;color:white;">Exercise 1.1</p>
<br>
In the next exercise we'll see an example of what's the difference between Path Planning and Obstacle Avoidance.
<br><br>
a) Open the Graphic Interface and execute the following command in order to start RViz.<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
rosrun rviz rviz<br>
</th>
</tr>
</table>
<br>

b) Execute the next command in order to launch the Obstacle Avoidance demo<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p><br>
roslaunch reem_navigation move_base_demo.launch<br>
</th>
</tr>
</table>
<br>

c) Execute the following command in the WebShell number #3 in order to spawn an obstacle in the room.<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #3</p><br>
rosrun gazebo_ros spawn_model -file /home/user/object.urdf -urdf -x 0 -y 0 -z 1 -model my_object<br>
</th>
</tr>
</table>

d) Properly congifure RViz in order to visualize both the global plan and the local plan.<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
Image of proper RViz configuration<br>
</th>
</tr>
</table>
<br>

e) Use the 2D Pose Estimate and Nav Goal tools in order to set an initial pose and send a goal to the robot.<br>

f) Set a goal that forces the robot to go past the obstacle you've just inserted in step c).<br>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 1.1</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1:</span>: <br>
<span style="color:orange">Note 2:</span>: <br>
<span style="color:orange">Note 2:</span>: <br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.1</p>

<img src="img/object.png" width="800"></img>

<img src="img/global_vs_local_path.jpg" width="800"></img>

<p style="background:#EE9023;color:white;">End of Exercise 1.1</p>

As you have seen in the previous exercise, the Global Plan and the Local Plan may not be the same one. Why do you think this can happen?
<br><br>
Well, the main reason is because they are based on different Costmaps, as we explained in the previous Unit (Chapter 4). 
<br><br>
<b>The Global Planner uses the Global CostMap in order to calculate the Global Plan.</b>
<br>
<b>The Local Planner, instead, uses the Local Costmap in order to calculate the Local Plan.</b>
<br>


### Local Costmap

Unlike the Global Costmap, which is based on the Static Map created in the Mapping unit (Chapter 2), this Local CostMap is continuously being updated with the robot's sensor data. This way, it'll be more precise than the Global Costmap, and it will also be able to detect any dinamic modification in the map that takes place once the robot has started its moving. For instance, a person walking into the environment.

#### Marking and Clearing

The costmap automatically subscribes to the sensor topics and updates itself according to the data it receives from them. Each sensor is used to either mark (insert obstacle information into the costmap), clear (remove obstacle information from the costmap), or both.<br> 
A marking operation is just an index into an array to change the cost of a cell.<br>
A clearing operation, however, cosists of raytracing through a grid from the origin of the sensor outwards for each observation reported.

#### Map Updates

The costmap performs map update cycles at a rate specified by the <i>update_frequency</i> parameter. Each cycle works as follows:
<ol>
<li>Sensor data comes in</li>
<li>Marking and clearing operations are performed</li>
<li>The appropied cost values are assigned as described above</li>
<li>obstacle inflation is performed on each cell with a LETHAL_OBSTACLE value. This consists of propagating cost values outwards from each occupied cell out to a specified inflation radius.</li>
</ol>

#### Inflation

Inflation is the process of propagating cost values out from occupied cells that decrease with distance. For this purpose, there are 5 specific symbols defined for costmap values:
<ul>
<li><b>"Lethal"</b> cost means that there is an actual obstacle in a cell. So if the robot center were in that cell, it would obviously be in collision.</li>
<li><b>"Inscribed"</b> cost means that a cell is less that the robot's inscribed radius away from an obstacle. So the robot would be in collision with an obstacle if the robot center is in a cell with this cost. </li>
<li><b>"Possibly Circumscribed"</b> cost is similar to inscrived, but using the robot's circumscribed radius as cutoff distance. Thus, if the robot center lies in a cell at or above this value, then it depends on the orientation of the robot whether it collides with an obstacle or not. </li>
<li><b>"Freespace"</b> cost is assumed to be zero, and it means that there is nothing that should keep the robot from going there. </li>
<li><b>"Unknown"</b> cost means there is no information about a given cell. The user of the costmap can interpret this as the want. </li>
</ul>

All other costs are assigned a value between "Freespace" and "Possibly Circumscribed" depending on their distance from a "Lethal" cell and the decay function provided by the user.

#### Topics

<ul>
<li><b>/laser_scan_topic (sensor_msgs/LaserScan)</b>: For each "LaserScan" source listed in the observation_sources parameter list, information from that source is used to update the costmap.</li>
<li><b>"map" (nav_msgs/OccupancyGrid)</b>: The costmap has the option of being initialized from a user-generated static mapa. If this option is selected, the costmap makes a service call to the map_server to obtan this map.</li>
</ul>

#### Parameters

<ul>
<li><b>/global_frame (string, default: "/map")</b>: The global frame for the costmap to operate in.</li>
<li><b>/robot_base_frame (string, default: "base_link")</b>: The name of the frame for the base link of the robot.</li>
<li><b>/transform_tolerance (double, default: 0.2)</b>: Specifies the delay in transform (tf) data that is tolerable in seconds.</li>
<li><b>/update_frequency (double, default: 5.0)</b>: The frequency in Hz for the map to be updated.</li>
<li><b>/publish_frequency (double, default: 0.0)</b>: The frequency in Hz for the map to be publish display information.</li>
<li><b>/rolling_window (bool, default: false)</b>: Whether or not to use a rolling window version of the costmap. If the static_map parameter is set to true, this parameter must be set to false.</li>
<li><b>/always_send_full_costmap (bool, default: false)</b>: If true the full costmap is published to "/grid" every update. If false only the part of the costmap that has changed is published on the "/grid_updates" topic.</li>
</ul>

Here you have an example of a local_costmap_params.yaml file:

In [ ]:
local_costmap:
   global_frame: odom
   robot_base_frame: /base_footprint
   update_frequency: 5.0
   publish_frequency: 2.0
   static_map: false
   rolling_window: true
   width: 4.0
   height: 4.0
   resolution: 0.05
   transform_tolerance: 0.5
   plugins:
    - {name: obstacle_layer,      type: "costmap_2d::VoxelLayer"}
    - {name: inflation_layer,     type: "costmap_2d::InflationLayer"}

<p style="background:#EE9023;color:white;">Exercise 1.2</p>
<br>
Open RViz and visualize the Local Costmap.
<br>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 1.2</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1:</span>: To see the costmap in Rviz, add a Map display.</i><br>
<span style="color:orange">Note 2:</span>: To see the local costmap set the topic to /move_base_node/local_costmap/costmap</b><br>
<span style="color:orange">Note 2:</span>: To see the global costmap set the topic to /move_base_node/global_costmap/costmap</b><br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.2</p><br>

<img src="img/local_costmap_rviz.png" width="500"></img>

<p style="background:#EE9023;color:white;">End of Exercise 1.2</p>

<p style="background:#EE9023;color:white;">Exercise 1.3</p>
<br>
Change some of the parameters introduced above and check how they affect to the Local Costamp.
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.3</p><br>

<p style="background:#EE9023;color:white;">End of Exercise 1.3</p>

### The Base Local Planner Package

This package provides a controller that drives a mobile base in the environment. Given a plan to follow (provided by the global-planner) and a costmap (provided by costmap-2D), it provides velocity commands in order to move the robot. Unlike the global planner, the local planner gets the odometry and the laser data, and finds a collision free local plan (let's imagine the local plan as a smaller part of the global plan) for the robot.

Along the way, the planner creates, at least locally around the robot, a value function, represented as a grid map. This value function encodes the costs of traversing through the grid cells. The controller's job is to use this value function to determine which velocities it has to send to the robot.

The basic idea of how this controller works is as follows:
<ol>
<li>Discretely sample in the robot's control space (dx,dy,dtheta)</li>
<li>For each sampled velocity, perform forward simulation from the robot's current state to predict what would happen if the sampled velocity were applied for some (short) period of time.</li>
<li>Evaluate (score) each trajectory resulting from the forward simulation, using a metric that incorporates characteristics such as: proximity to obstacles, proximity to the goal, proximity to the global path, and speed. Discard illegal trajectories (those that collide with obstacles).</li>
<li>Pick the highest-scoring trajectory and send the associated velocity to the mobile base.</li>
<li>Rinse and repeat.</li>
</ol>

#### Topics involved

<ul>
<li><b>/global_plan (nav_msgs/Path)</b>: The portion of the global plan that the local planner is currently attempting to follow. Used primarily for visualization purposes.</li>
<li><b>/local_plan (nav_msgs/Path)</b>: The local plan or trajectory that scored the highest on the last cycle. Used primarily for visualization purposes.</li>
<li><b>/cost_cloud (sensor_msgs/PointCloud2)</b>: The cost grid used for planning. Used for visualization purposes. See the publish_cost_grid_pc parameter for enabling/disabling this visualization.</li>
</ul>

<p style="background:#EE9023;color:white;">Exercise 1.4</p>
<br>
Open Rviz and a add a Display in order to be able to see the topics introduced above.

<br>
Subscribe to the topics where the Local Planner publishes, and have a look at them.
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.4</p><br>

<img src="img/planned_path.png" width="300"></img>

<p style="background:#EE9023;color:white;">End of Exercise 1.4</p>

<p style="background:#EE9023;color:white;">Exercise 1.5</p>
<br>
Open Rviz and a add a Display in order to be able to see both the Global and Local Plan.

<br>
Subscribe to the topics where the Local Planner publishes, and have a look at them.
<br>

Execute the next command in order to insert an object in the middle of the room. See what happens<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p><br>
insert object
<br>
</th>
</tr>
</table>
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.5</p><br>

<img src="img/Obstacle.png" width="300"></img>

<p style="background:#EE9023;color:white;">End of Exercise 1.5</p>

As always, The base local planner has many parameters in order to customize its behavior. Let's check some of this parameters:

#### Robot Configuration Parameters

#### Goal Tolerance Parameters

#### Forward Simulation Parameters

#### Trajectory Scoring Parameters

#### Oscillation Prevention Parameters

#### Global Plan Parameters

<p style="background:#EE9023;color:white;">Exercise 1.8</p>
<br>
Execute the command used in the Exercise 1.1, but modify some of this parameters in order to see how they affect to the Local Path Planning.
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.8</p><br>

Different Local Plans such as in the Pictures below

<p style="background:#EE9023;color:white;">End of Exercise 1.8</p>

## The DWA Local Planner Package

This package provides an implementation of the Dynamic Window Approach to local robot navigation on a plane. Given a global plan to follow and a costmap, the local planner produces velocity commands to send to a mobile base. This package supports any robot who's footprint can be represented as a convex polygon or cicrle, and exposes its configuration as ROS parameters that can be set in a launch file. The parameters for this planner are also dynamically reconfigurable.

DWA (Dynamic Window Algorithm) differs from Trajectory Rollout in how the robot's control space is sampled. Trajectory Rollout samples from the set of achievable velocities over the entire forward simulation period given the acceleration limits of the robot, while DWA samples from the set of achievable velocities just for one simulation step given the acceleration limits of the robot.
<br><br>
DWA is a more efficient algorithm because it samples a smaller space, but may

Here you have an example of the dwa_local_planner_params.yaml:

In [ ]:
DWAPlannerROS:

# Robot Configuration Parameters - Kobuki
  max_vel_x: 0.5  # 0.55
  min_vel_x: 0.0

  max_vel_y: 0.0  # diff drive robot
  min_vel_y: 0.0  # diff drive robot

  max_trans_vel: 0.5 # choose slightly less than the base's capability
  min_trans_vel: 0.1  # this is the min trans velocity when there is negligible rotational velocity
  trans_stopped_vel: 0.1

  # Warning!
  #   do not set min_trans_vel to 0.0 otherwise dwa will always think translational velocities
  #   are non-negligible and small in place rotational velocities will be created.

  max_rot_vel: 5.0  # choose slightly less than the base's capability
  min_rot_vel: 0.4  # this is the min angular velocity when there is negligible translational velocity
  rot_stopped_vel: 0.4

  acc_lim_x: 1.0 # maximum is theoretically 2.0, but we
  acc_lim_theta: 2.0
  acc_lim_y: 0.0      # diff drive robot

# Goal Tolerance Parameters
  yaw_goal_tolerance: 0.3  # 0.05
  xy_goal_tolerance: 0.15  # 0.10
  # latch_xy_goal_tolerance: false

# Forward Simulation Parameters
  sim_time: 1.0       # 1.7
  vx_samples: 6       # 3
  vy_samples: 1       # diff drive robot, there is only one sample
  vtheta_samples: 20  # 20

# Trajectory Scoring Parameters
  path_distance_bias: 64.0      # 32.0   - weighting for how much it should stick to the global path plan
  goal_distance_bias: 24.0      # 24.0   - wighting for how much it should attempt to reach its goal
  occdist_scale: 0.5            # 0.01   - weighting for how much the controller should avoid obstacles
  forward_point_distance: 0.325 # 0.325  - how far along to place an additional scoring point
    stop_time_buffer: 0.2         # 0.2    - amount of time a robot must stop in before colliding for a valid traj.
  scaling_speed: 0.25           # 0.25   - absolute velocity at which to start scaling the robot's footprint
  max_scaling_factor: 0.2       # 0.2    - how much to scale the robot's footprint when at speed.

# Oscillation Prevention Parameters
  oscillation_reset_dist: 0.05  # 0.05   - how far to travel before resetting oscillation flags

# Debugging
  publish_traj_pc : true
  publish_cost_grid_pc: true
  global_frame_id: odom


# Differential-drive robot configuration - necessary?
#  holonomic_robot: false

### Recovery Behaviors

<img src="img/recovery_behaviors.png" width="700"></img>

Running the move_base node on a robot that is properly configured (please see navigation stack documentation for more details) results in a robot that will attempt to achieve a goal pose with its base to within a user-specified tolerance. In the absence of dynamic obstacles, the move_base node will eventually get within this tolerance of its goal or signal failure to the user. The move_base node may optionally perform recovery behaviors when the robot perceives itself as stuck. By default, the move_base node will take the following actions to attempt to clear out space:

First, obstacles outside of a user-specified region will be cleared from the robot's map. Next, if possible, the robot will perform an in-place rotation to clear out space. If this too fails, the robot will more aggressively clear its map, removing all obstacles outside of the rectangular region in which it can rotate in place. This will be followed by another in-place rotation. If all this fails, the robot will consider its goal infeasible and notify the user that it has aborted. These recovery behaviors can be configured using the recovery_behaviors parameter, and disabled using the recovery_behavior_enabled parameter.

### The Rotate Recovery Package

This package provides a simple recovery behavior that attempts to clear out space by rotating the robot 360 degrees  if local obstacles are found in the way.

It has some parameters you can customize in order to change or improve its behavior:

#### Rotate Recovery Parameters

<ul>
<li><b>/sim_granularity (double, default: 0.017)</b>: The distance in radians between checks for obstacles when checking if an in-place rotation is safe. Defaults to 1 degree.</li>
<li><b>/frequency (double, default: 20.0)</b>: The frequency in HZ at which to send velocity commands to the mobile base.</li>
</ul>

#### TrajectoryPlanner Parameters

<p style="color:red;"> IMPORTANT: These parameters are already set when using the base_local_planner local planner, they only need to be set explicitly for the recovery behavior if a different local planner is used.</p>

<ul>
<li><b>/yaw_goal_tolerance (double, default: 0.05)</b>: The tolerance in radians for the controller in yaw/rotation when achieving its goal</li>
<li><b>/acc_lim_th (double, default: 3.2)</b>: The rotational acceleration limit of the robot in radians/sec^2</li>
<li><b>/max_rotational_vel (double, default: 1.0)</b>: The maximum rotational velocity allowed for the base in radians/sec</li>
<li><b>/min_in_place_rotational_vel (double, default: 0.4)</b>: The minimum rotational velocity allowed for the base while performing in-place rotations in radians/sec</li>
</ul>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.9</p><br>

Gif of Robot Rotating

<p style="background:#EE9023;color:white;">End of Exercise 1.9</p>

### The Clear Costmap Recovery package

This package provides a simple recovery behavior that clears out space by clearing obstacles outside of a specified region from the robot's map.

The radius away from the robot (in meters) which obstacles will be removed from the costmap when they are reverted to the static map, can be setted by modifying the next parameter:
<ul>
<li><b>/reset_distance</b></li>
</ul>

## How Obstacle Avoidance works

### Penalty terms

Obstacle avoidance is achieved as part of the overall trajectory optimization. Obviously, optimization is concerned with finding a minimum cost solution (trajectory) of a specified cost function (objective function). Put simply: If a planned (future) pose violates a desired separation from obstacles, the costs of the cost function must therefore increase. Ideally, the cost function value must be infinity in those cases or the optimizer might better reject those areas completely. However, this would require optimizers handling hard constraints (resp. solving nonlinear programs). The teb_local_planner waives the ability of taking hard constraints into account in order to better account for efficiency. Hard constraints are transformed into soft constraints, leading to the composition of quadratic penalty terms with finite costs. 

<img src="img/obstacle_soft_constraint.png" width="400"></img>

The figure above shows an examplary penalty term (for obstacle avoidance). The allowed minimum Euclidean distance to the obstacle (parameter min_obstacle_dist) is set to 0.2 meters. Therefore, distances below 0.2m lead to a non-zero cost. Now imagine that the optimization problem consists of many more cost terms. Some of them are conflicting such as time-optimality. Therefore it might happen, that the optimizer takes a small violation (and therefore small penalty) into account in order to minimize the overall composed cost function. Here you have two options in order to tune the behavior: 

<ol>
<li>Adjust the optimization weights (scaling of the individual costs, here it is parameter weight_obstacle). But if you choose too high values, the optimization problem becomes ill-conditioned leading to a poor convergance behavior. </li>
<li>Shift the argument by adding an "extra margin". By adding a small extra margin to the min_obstacle_dist parameter you implicitly increase the cost value at 0.2m. You might shift all penalty terms at once with the single parameter penalty_epsilon, but be careful since doing so will massively influence the optimziation results.</li>
</ol>

### Locally optimal solutions

Notice, the optimizer on its own only finds locally optimal solutions. Imagine, the robot might be encapsulated laterally by two obstacles. The penalty terms are indeed non-zero, but the optimizer get stucked (to this local minimum) since moving the corresponding pose laterally towards one of the obstacles would further increase the total cost. You can try that easily with the test_optim_node (see Tutorial Set up and test Optimization and turn off homotopy class planning). The behavior should be similar to the one in the following figure: 

<img src="img/local_minimum_two_obstacles.png" width="600"></img>

The trajectory cannot jump over the obstacles. Even the poses itselves are pushed away from the region between the obstacles (red arrows). Obviously, this circumstance should be avoided in practice. Hence, the homotopy class planning algorithm seeks for (topologically) alternative solutions and the feasibility check (see section below) rejects such a solution before actually commanding the robot. 



### Association between Poses and Obstacles

The following figure depicts a snapshot of a common planning scenario: 



<img src="img/robot_traj_obst.png" width="600"></img>

The scenario consists of a mobile robot which approaches a polygon-shaped obstacle while traveling to the current goal. The planned (discrete) trajectory is composed of multiple robot poses. The planner aims to arrange each two consecutive poses according to the desired temporal resolution (Parameter dt_ref). Note, the actual resolution is not fixed/freezed since the optimizer needs to adjust the transition time in order to seek for time-optimality. 

For obstacle avoidance, the distance between a planned pose and the obstacle has to be bounded from below. The example trajectory in the figure consists of 8 changeable poses (start and goal poses are fixed). You might agree, that many distance calculations are required in order to achieve a collision free trajectory (the optimizer invokes the computation of cost function values many times). In order to speed up the optimization, a dedicated association strategy is implemented. 

For each obstacle (point/ occupied costmap cell, line, polygon) the closest pose of the planned trajectory is located (see figure above). According to the value of parameter obstacle_poses_affected neighbors of the closest pose are taken into account as well. Only this selected subset of poses is taken into account in the subsequent optimization step (here 3 poses and therefore 3 penalty terms). The association process is repeated after no_inner_iterations (parameter), resp. within each outer optimization iteration. The value of obstacle_poses_affected slightly influences the smoothness of the trajectory around obstacles. Also bigger obstacles require more connected poses to avoid inadmissable shortcuts. You may also choose a high value (> trajectory length) in order to connect all poses with each obstacle. 

Note, the robot footprint model is taken into account for distance calculations and is therefore crucial for the required computational resources. Details are summarized in the following section. 

## Robot Footprint Model

The robot footprint model approximates the robots' 2D contour for optimization purposes. The model is crucial for the complexity of distance calculations and hence for the computation time. Therefore, the robot footprint model constitutes a dedicated parameter instead of loading the footprint from the common costmap_2d parameters. The optimization footprint model might differ from the costmap footprint model (which is instead used for the feasibility check, see section below). 

The footprint model is selected and configured using the parameter server. You can add the following parameter structure to your teb_local_planner config file: 

The default footprint model is of type "point". 

Note, the footprint is published to ~<name>/teb_markers and can be visualized in rviz (e.g. for verification).

Important: For car-like robots, the pose [0,0] is located at the rear-axle (axis of rotation)! 

All different types are described in the following paragraphs: 



### Footprint Type: Point

The robot is modeled as a single point. For this type the least computation time is required. 

### Footprint Type: Circular

The robot is modeled as a simple circle with a given radius ~/footprint_model/radius. The distance calculation is similar to the one of the point-type robot, but with the exepction, that the robots' radius is added to the parameter min_obstacle_dist each function call. You can ged rid of this extra addition by chosing a point-type robot and adding the radius to the minimum obstacle distance a-priori. 

### Footprint Type: Line

The line robot is useful for robots that exhibit different expansions/lengths in the longitudinal and lateral directions. The line (segment) can be configured using parameters ~/footprint_model/line_start and ~/footprint_model/line_end ([x,y] coordinates each). The robot (axis of rotation) is assumed to be at [0,0] (Unit: meters). Make sure to encapsulate the complete robot by further adjustment of the parameter min_obstacle_dist (see following example). 

<img src="img/line_model.png" width="300"></img>

### Footprint Type: Two Circles

Another possibility to approximate the robot's contour consists of defining two circles. Each circle is described by an offset along the robots' x-axis and a radius: ~/footprint_model/front_offset, ~/footprint_model/front_radius, ~/footprint_model/rear_offset and ~/footprint_model/rear_radius. Offsets may be negative. 

Refer to the following figure as an example: 

<img src="img/two_circles_model.png" width="400"></img>

### Footprint Type: Polygon

A complex model can be incorporated by defining a closed polygon. The polygon is defined in terms of a list of vertices (provide x and y coordinates for each vertex). The robots axis of rotation is assumed to be located at [0,0] (Unit: meters). Please do not repeat the first vertex since the polygon is closed automatically. 

Take in mind, that each additional edge significantly increases the required computation time! You may copy your footprint model from your costmap common parameter file. 

## Feasibility Check

A feasibility check is performed after the optimizer returns a trajectory and before the velocity commands are sent to the robot. The purpose of this check is to identify an invalid/infeasible trajectory that might be produced by the optimizer (just remember: soft constraints, local minima, etc.). 

Currently, the algorithm iterates the first n poses (n = ~/feasibility_check_no_poses (parameter!)) starting from the current robot pose and checks whether the those poses are collision free. For detecting if a collision occurs, the costmap footprint is used (see Navigation tutorials)! Therefore, this validation-model might be more complex than the footprint used for optimization (see section above). 

The value ~/feasibility_check_no_poses should not be chosen too high, since the optimizer might not be converged completely: Figuratively, small obstacle violations in the (far) future could be corrected while the robot is moving towards the goal. 

If you are driving in narrow environments, make sure to configure the obstacle avoidance behavior (local planner and global planner) properly. Otherwise, the local planner might reject an infeasible trajectory (from its point of view), but the global planner in contrast could further think that the selected (global) plan is feasible: the robot could get stucked. 